# Pearson Dfs EGFR proteomics vs all proteins 

This notebook records the pearson correlation stats for EGFR proteomics vs all proteomics. It records only the FDR Benjamini/Hochberg significant comparisons. 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest


import cptac
import cptac.utils as u
import plot_utils as p


/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
def create_prot_list(df):
    df = df.loc[:,~df.columns.duplicated()]
    prot_genes_list = df.columns.values.tolist()
    prot_genes_list.remove('EGFR_Mutation')
    prot_genes_list.remove('EGFR_Location')
    prot_genes_list.remove('EGFR_Mutation_Status')
    prot_genes_list.remove('Sample_Status')
    return prot_genes_list

In [4]:
#load cptac data 
brain = cptac.Gbm()
kidney = cptac.Ccrcc()
Ovar = cptac.Ovarian()
colon = cptac.Colon()
brca = cptac.Brca()
luad = cptac.Luad()
Hnscc = cptac.Hnscc()
Lscc = cptac.Lscc()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 2)


cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 7)


cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 8)


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 9)


# Step 1 Create Data frames 

For each cancer type,
load proteomic data and mutation data for EGFR and
filter out non-tumor samples 

# Step 2  Create list of Proteins for Comparison

For each cancer type, create list of proteins by using create_prot_list function which extracts column names and removes extra columns.

# Step 3 Run Pearson Correlation Function  

Run function on df and compare EGFR proteomics to all other proteins in protein list. Store only the significant comparisons. Save df as csv file

# GBM

In [7]:
df1 = brain.join_omics_to_mutations(omics_df_name="proteomics", mutations_genes="EGFR")
df1_tumor = df1.loc[df1['Sample_Status'] == "Tumor"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 93 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [8]:
prot_genes_list = create_prot_list(df1_tumor)

In [9]:
df2= p.wrap_pearson_corr(df1_tumor,"EGFR_proteomics",comparison_columns= prot_genes_list,correction_method='fdr_bh', return_all = False)
#df2.to_csv("csv_files/GBM_EGFR_all_pearson_FDR.csv")
df2['Cancer Type']='GBM'
#df2.to_csv("csv_files/GBM_EGFR_all_pearson_FDR2.csv")
df2

,Comparison,Correlation,P_value,Cancer Type
454,EGFR_proteomics,1.000000,0.000000e+00,GBM
1043,PHLDA1_proteomics,0.816848,6.553435e-25,GBM
642,GRB2_proteomics,-0.610889,1.886384e-11,GBM
1353,SOCS2_proteomics,0.562720,1.343464e-09,GBM
274,CDH4_proteomics,0.559180,1.790048e-09,GBM
...,...,...,...,...
1511,USP2_proteomics,0.280545,7.399884e-03,GBM
1536,WAPL_proteomics,-0.267631,7.403862e-03,GBM
1324,SLC25A42_proteomics,0.267588,7.413549e-03,GBM
523,FANCD2_proteomics,0.278945,7.416331e-03,GBM


In [10]:
df2['Rank'] = np.arange(len(df2))
df2

,Comparison,Correlation,P_value,Cancer Type,Rank
454,EGFR_proteomics,1.000000,0.000000e+00,GBM,0
1043,PHLDA1_proteomics,0.816848,6.553435e-25,GBM,1
642,GRB2_proteomics,-0.610889,1.886384e-11,GBM,2
1353,SOCS2_proteomics,0.562720,1.343464e-09,GBM,3
274,CDH4_proteomics,0.559180,1.790048e-09,GBM,4
...,...,...,...,...,...
1511,USP2_proteomics,0.280545,7.399884e-03,GBM,1585
1536,WAPL_proteomics,-0.267631,7.403862e-03,GBM,1586
1324,SLC25A42_proteomics,0.267588,7.413549e-03,GBM,1587
523,FANCD2_proteomics,0.278945,7.416331e-03,GBM,1588


# Kidney 

In [11]:
df1 = kidney.join_omics_to_mutations(omics_df_name="proteomics",mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(1)
df1_tumor = df1.loc[df1['Sample_Status'] == "Tumor"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 193 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [12]:
prot_genes_list = create_prot_list(df1_tumor)

In [13]:
df2=p.wrap_pearson_corr(df1_tumor,"EGFR_proteomics",comparison_columns= prot_genes_list,correction_method='fdr_bh',return_all = False)
df2.to_csv("csv_files/Kidney_EGFR_all_pearson_FDR.csv")
df2['Cancer Type']='Kidney'
df2.to_csv("csv_files/Kidney_EGFR_all_pearson_FDR2.csv")
df2

,Comparison,Correlation,P_value,Cancer Type
272,EGFR_proteomics,1.000000,0.000000e+00,Kidney
103,ATP11A_proteomics,0.577079,4.144636e-11,Kidney
699,PLCB1_proteomics,0.561952,1.684847e-10,Kidney
71,APAF1_proteomics,0.560909,1.851119e-10,Kidney
410,HPCAL1_proteomics,0.556681,2.701940e-10,Kidney
...,...,...,...,...
783,RAP1GAP2_proteomics,-0.273992,5.561371e-03,Kidney
310,FAF2_proteomics,0.262631,5.573690e-03,Kidney
15,ACP5_proteomics,-0.262417,5.613804e-03,Kidney
844,SFRP1_proteomics,-0.262301,5.635779e-03,Kidney


In [14]:
df2['Rank'] = np.arange(len(df2))
df2

,Comparison,Correlation,P_value,Cancer Type,Rank
272,EGFR_proteomics,1.000000,0.000000e+00,Kidney,0
103,ATP11A_proteomics,0.577079,4.144636e-11,Kidney,1
699,PLCB1_proteomics,0.561952,1.684847e-10,Kidney,2
71,APAF1_proteomics,0.560909,1.851119e-10,Kidney,3
410,HPCAL1_proteomics,0.556681,2.701940e-10,Kidney,4
...,...,...,...,...,...
783,RAP1GAP2_proteomics,-0.273992,5.561371e-03,Kidney,1068
310,FAF2_proteomics,0.262631,5.573690e-03,Kidney,1069
15,ACP5_proteomics,-0.262417,5.613804e-03,Kidney,1070
844,SFRP1_proteomics,-0.262301,5.635779e-03,Kidney,1071


# Ovarian 

In [15]:
df1 = Ovar.join_omics_to_mutations(omics_df_name="proteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(1)
df1_tumor = df1.loc[df1['Sample_Status'] == "Tumor"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 102 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [16]:
prot_genes_list = create_prot_list(df1_tumor)

In [17]:
df2= p.wrap_pearson_corr(df1_tumor,"EGFR_proteomics",comparison_columns= prot_genes_list,correction_method='fdr_bh', return_all = False)
df2.to_csv("csv_files/Ovar_EGFR_all_pearson_FDR.csv")
df2['Cancer Type']='Ovarian'
df2.to_csv("csv_files/Ovar_EGFR_all_pearson_FDR2.csv")


In [18]:
df2['Rank'] = np.arange(len(df2))
df2

,Comparison,Correlation,P_value,Cancer Type,Rank
49,EGFR_proteomics,1.000000,0.000000e+00,Ovarian,0
52,ERO1A_proteomics,0.512158,7.452351e-07,Ovarian,1
32,CGGBP1_proteomics,-0.492993,2.192339e-06,Ovarian,2
116,PROSER2_proteomics,0.524593,2.244019e-06,Ovarian,3
39,CP_proteomics,0.489328,2.675011e-06,Ovarian,4
...,...,...,...,...,...
9,ANKRD1_proteomics,-0.422420,8.600168e-04,Ovarian,174
96,MID1_proteomics,0.358764,8.667758e-04,Ovarian,175
58,FAM57A_proteomics,0.523408,8.866811e-04,Ovarian,176
101,NAP1L1_proteomics,-0.357673,9.011982e-04,Ovarian,177


# Colon

In [19]:
df1 = colon.join_omics_to_mutations(omics_df_name="proteomics", mutations_genes="EGFR")
df1_tumor = df1.loc[df1['Sample_Status'] == "Tumor"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 204 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: 01CO001, 05CO005, 05CO007, 05CO014, 05CO055, 11CO059, 11CO060, 14CO002, 16CO012, 24CO005 (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [20]:
prot_genes_list = create_prot_list(df1_tumor)

In [21]:
df2= p.wrap_pearson_corr(df1_tumor,"EGFR_proteomics",comparison_columns= prot_genes_list, correction_method='fdr_bh',return_all = False)
df2.to_csv("csv_files/Colon_EGFR_all_pearson_FDR.csv")
df2['Cancer Type']='Colon'
df2.to_csv("csv_files/Colon_EGFR_all_pearson_FDER2.csv")
df2

,Comparison,Correlation,P_value,Cancer Type
145,EGFR_proteomics,1.000000,0.000000e+00,Colon
586,UTP6_proteomics,-0.510343,1.733748e-07,Colon
425,RAE1_proteomics,-0.498802,1.993818e-07,Colon
178,GAR1_proteomics,-0.489788,3.550550e-07,Colon
428,RALYL_proteomics,-0.485763,4.569947e-07,Colon
...,...,...,...,...
541,TIMP1_proteomics,0.287791,4.257833e-03,Colon
85,CCDC6_proteomics,0.287735,4.265624e-03,Colon
259,LAMA5_proteomics,0.287734,4.265767e-03,Colon
516,SSR2_proteomics,0.301814,4.266608e-03,Colon


In [22]:
df2['Rank'] = np.arange(len(df2))
df2

,Comparison,Correlation,P_value,Cancer Type,Rank
145,EGFR_proteomics,1.000000,0.000000e+00,Colon,0
586,UTP6_proteomics,-0.510343,1.733748e-07,Colon,1
425,RAE1_proteomics,-0.498802,1.993818e-07,Colon,2
178,GAR1_proteomics,-0.489788,3.550550e-07,Colon,3
428,RALYL_proteomics,-0.485763,4.569947e-07,Colon,4
...,...,...,...,...,...
541,TIMP1_proteomics,0.287791,4.257833e-03,Colon,608
85,CCDC6_proteomics,0.287735,4.265624e-03,Colon,609
259,LAMA5_proteomics,0.287734,4.265767e-03,Colon,610
516,SSR2_proteomics,0.301814,4.266608e-03,Colon,611


# Brca 

In [23]:
df1 = brca.join_omics_to_mutations(omics_df_name="proteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(1)
df1_tumor = df1.loc[df1['Sample_Status'] == "Tumor"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 118 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [24]:
prot_genes_list = create_prot_list(df1_tumor)

In [25]:
df2= p.wrap_pearson_corr(df1_tumor,"EGFR_proteomics",comparison_columns= prot_genes_list, correction_method='fdr_bh',return_all = False)
df2.to_csv("csv_files/Brca_EGFR_all_pearson_FDR.csv")
df2['Cancer Type']='Brca'
df2.to_csv("csv_files/Brca_EGFR_all_pearson_FDR2.csv")

In [26]:
df2['Rank'] = np.arange(len(df2))
df2

,Comparison,Correlation,P_value,Cancer Type,Rank
703,EGFR_proteomics,1.000000,0.000000e+00,Brca,0
1419,MPP6_proteomics,0.648202,4.834438e-15,Brca,1
538,CPNE8_proteomics,0.616453,2.229263e-13,Brca,2
553,CRYBG3_proteomics,0.609997,4.612176e-13,Brca,3
1858,PSAT1_proteomics,0.609319,4.973595e-13,Brca,4
...,...,...,...,...,...
924,GGH_proteomics,0.229847,1.347031e-02,Brca,2664
1182,KIF15_proteomics,0.229837,1.347467e-02,Brca,2665
1407,MNS1_proteomics,0.270088,1.353485e-02,Brca,2666
2058,RRP1_proteomics,0.229647,1.355376e-02,Brca,2667


# LUAD

In [27]:
df1 = luad.join_omics_to_mutations(omics_df_name="proteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(1)
df1_tumor = df1.loc[df1['Sample_Status'] == "Tumor"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 171 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [28]:
prot_genes_list = create_prot_list(df1_tumor)

In [29]:
df2= p.wrap_pearson_corr(df1_tumor,"EGFR_proteomics",comparison_columns= prot_genes_list,correction_method='fdr_bh', return_all = False)
df2.to_csv("csv_files/Luad_EGFR_all_pearson_FDR.csv")
df2['Cancer Type']='Luad'
df2.to_csv("csv_files/Luad_EGFR_all_pearson_FDR2.csv")

In [30]:
df2['Rank'] = np.arange(len(df2))
df2

,Comparison,Correlation,P_value,Cancer Type,Rank
332,EGFR_proteomics,1.000000,0.000000e+00,Luad,0
434,GGCT_proteomics,0.609006,1.676601e-12,Luad,1
563,LANCL2_proteomics,0.588808,1.328507e-11,Luad,2
699,NUDCD3_proteomics,0.568383,9.362813e-11,Luad,3
1034,TAX1BP1_proteomics,0.561554,1.746498e-10,Luad,4
...,...,...,...,...,...
1130,VGLL4_proteomics,0.261583,5.773032e-03,Luad,1193
1001,ST5_proteomics,0.261481,5.792846e-03,Luad,1194
261,CSTF1_proteomics,0.261472,5.794461e-03,Luad,1195
986,SPRYD4_proteomics,-0.261441,5.800620e-03,Luad,1196


# HNSCC

In [31]:
df1 = Hnscc.join_omics_to_mutations(omics_df_name="proteomics",  mutations_genes="EGFR")
df1_tumor = df1.loc[df1['Sample_Status'] == "Tumor"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 174 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [32]:
prot_genes_list = create_prot_list(df1_tumor)

In [33]:
df2= p.wrap_pearson_corr(df1_tumor,"EGFR_proteomics",comparison_columns= prot_genes_list,correction_method='fdr_bh', return_all = False)
df2.to_csv("csv_files/Hnscc_EGFR_all_pearson_FDr.csv")
df2['Cancer Type']='Hnscc'
df2.to_csv("csv_files/Hnscc_EGFR_all_pearson_FDR2.csv")

In [34]:
df2['Rank'] = np.arange(len(df2))
df2

,Comparison,Correlation,P_value,Cancer Type,Rank
280,EGFR_proteomics,1.000000,0.000000e+00,Hnscc,0
522,LANCL2_proteomics,0.796728,3.798194e-25,Hnscc,1
689,PHLDA3_proteomics,0.763784,4.542521e-22,Hnscc,2
687,PHLDA1_proteomics,0.664271,3.426615e-15,Hnscc,3
688,PHLDA2_proteomics,0.649034,2.302129e-14,Hnscc,4
...,...,...,...,...,...
404,GPD1L_proteomics,-0.266473,5.097373e-03,Hnscc,1069
880,SELL_proteomics,-0.266360,5.116897e-03,Hnscc,1070
563,MBD1_proteomics,-0.266098,5.162417e-03,Hnscc,1071
848,RRP12_proteomics,0.266066,5.167833e-03,Hnscc,1072


# Lscc

In [35]:
df1 = Lscc.join_omics_to_mutations(omics_df_name="proteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(1)
df1_tumor = df1.loc[df1['Sample_Status'] == "Tumor"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 208 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [36]:
prot_genes_list = create_prot_list(df1_tumor)

In [37]:
df2= p.wrap_pearson_corr(df1_tumor,"EGFR_proteomics",comparison_columns= prot_genes_list,correction_method='fdr_bh', return_all = False)
df2.to_csv("csv_files/Lscc_EGFR_all_pearson_FDR.csv")
df2['Cancer Type']='Lscc'
df2.to_csv("csv_files/Lscc_EGFR_all_pearson_FDR2.csv")

In [38]:
df2['Rank'] = np.arange(len(df2))
df2

,Comparison,Correlation,P_value,Cancer Type,Rank
138,EGFR_proteomics,1.000000,0.000000e+00,Lscc,0
307,PHLDA1_proteomics,0.709812,3.919195e-18,Lscc,1
309,PHLDA3_proteomics,0.685644,1.412309e-16,Lscc,2
247,LANCL2_proteomics,0.572261,6.528910e-11,Lscc,3
210,ITGB4_proteomics,0.567380,1.026981e-10,Lscc,4
...,...,...,...,...,...
42,ATXN10_proteomics,0.290219,2.101111e-03,Lscc,469
248,LIMA1_proteomics,0.290115,2.109252e-03,Lscc,470
6,ADA2_proteomics,-0.289977,2.120119e-03,Lscc,471
356,RFXAP_proteomics,-0.289937,2.123227e-03,Lscc,472
